In [7]:
import json
import os
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display

from ds_contrib.tools.io.gscloud import GSBrowser

In [8]:
## | hide


CONFIGS_DIR = Path("../../configs/storage/gscloud/")
ENV_DIR = Path("../../configs/env/local/")

with open(CONFIGS_DIR / "projects_vars.json") as f:
    projects = json.load(f)

# choose project
project = projects["dev"]
env_path = Path(ENV_DIR / f'{project["env"]}_roadly.env')

_ = load_dotenv(env_path)  # read local .env file
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
roadly_cookie = os.getenv("ROADLY_COOKIE")
print(f"Initial configuration has finished:\nProject: {project}")

Initial configuration has finished:
Project: {'project': 'roadly-project-dev', 'env': 'dev', 'coldline_name': 'standard'}


In [9]:
def get_reclslam_df(recslam_dir: Path, structure: dict):
    return pd.DataFrame.from_records(
        [(k, v["desc"], recslam_dir / v["path"]) for k, v in structure.items()],
        index="name",
        columns=["name", "desc", "path"],
    )

1. Choose reclslam path
2. Download video and meta
3. Split video to frames
4. Run video browser
5. Export dataset to csv


In [10]:
sampled_datasets = pd.read_csv("/Users/arseniy/Downloads/выборка.csv")
items = sampled_datasets["name"].sort_values()

In [12]:
def create_items_dropdown(items):
    """Create a dropdown widget with files in the specified directory."""
    # List all files in the directory
    # Create a dropdown widget with the files
    dropdown = widgets.Dropdown(
        options=items,
        description="Files:",
        disabled=False,
    )

    # Display the widget
    display(dropdown)

    return dropdown

In [13]:
# Call the function with your desired directory
dropdown = create_items_dropdown(items)

Dropdown(description='Files:', options=('2021-11-21_15-17-06_4453', '2022-02-04_14-45-39_4453', '2022-02-04_14…

In [26]:
chosen_video = sampled_datasets[sampled_datasets["name"] == dropdown.value][
    "video_url"
].values[0]

In [27]:
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
browser = GSBrowser(project=project["project"], credentials=google_app_creds)

In [28]:
browser.download_file(chosen_video)

INFO:ds_contrib.core.paths:Created temporary directory for downloads: `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpj8b5e_sx`
INFO:ds_contrib.tools.io.gscloud:Downloading file from URI: `Highway_Drives/2022-05-07_05-30-22_FA04EAE3-A6A/video_2` to path: `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpj8b5e_sx/roadly-dev-standard-videos/Highway_Drives/2022-05-07_05-30-22_FA04EAE3-A6A/video_2`
INFO:google.resumable_media._helpers:No MD5 checksum was returned from the service while downloading https://storage.googleapis.com/download/storage/v1/b/roadly-dev-standard-videos/o/Highway_Drives%2F2022-05-07_05-30-22_FA04EAE3-A6A%2Fvideo_2?generation=1685751361077352&alt=media
(which happens for composite objects), so client-side content integrity
checking is not being performed.


In [29]:
from ds_contrib.core.paths import list_paths


LOCAL_RECSLAM_PATH = list(list_paths(browser.downloads_path, recursive=True))[0].parent

In [30]:
from ds_contrib.core.data.video import sample_frames_from_video


sample_frames_from_video(
    LOCAL_RECSLAM_PATH / "video_2", LOCAL_RECSLAM_PATH / "frames", time_delta=2
)

Sampling frames:   0%|          | 0/303 [00:00<?, ?it/s]

In [31]:
from ds_contrib.core.paths import list_paths
from ds_contrib.tools.browser import ImageBrowser


ib = ImageBrowser(
    list(list_paths(LOCAL_RECSLAM_PATH / "frames", recursive=True, sort=True))
)
ib.browse()

In [48]:
browser.downloads_dir.cleanup()

INFO:ds_contrib.core.paths:Cleaning up temporary directory `/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmprx34dhgl`
